<a href="https://colab.research.google.com/github/Du-nara/ME421-Mechanical-Systems-Lab-A3/blob/main/Vibration/E_20_388_Vibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 2-DOF vibration model for TecQuipment rigid beam system
# DOF 1: rigid-body rotation (theta)
# DOF 2: first bending modal coordinate (eta)

import sympy as sp

# ------------------------------------------------------------------
# 1. Define symbolic parameters
# ------------------------------------------------------------------
L, rho, A, EI = sp.symbols('L rho A EI', positive=True)     # Beam properties
k = sp.symbols('k', positive=True)                           # Spring stiffness
m_m, x_m = sp.symbols('m_m x_m', positive=True)              # Motor mass and location

# Generalized coordinates
theta, eta = sp.symbols('theta eta')
theta_dd, eta_dd = sp.symbols('theta_dd eta_dd')

# Spatial variable
x = sp.symbols('x')

# ------------------------------------------------------------------
# 2. Assumed mode shape (first bending mode, pinned-free approx.)
# (Simple polynomial admissible function for lab-level modelling)
# ------------------------------------------------------------------
phi = x**2 * (3*L - x)   # admissible bending mode shape

# ------------------------------------------------------------------
# 3. Kinetic Energy (Beam)
# w(x,t) = x*theta + phi(x)*eta
# ------------------------------------------------------------------
w_dot = x*sp.symbols('theta_d') + phi*sp.symbols('eta_d')

T_beam = sp.integrate(0.5 * rho * A * w_dot**2, (x, 0, L))

# Extract mass matrix terms
M11_b = sp.integrate(rho * A * x**2, (x, 0, L))
M12_b = sp.integrate(rho * A * x * phi, (x, 0, L))
M22_b = sp.integrate(rho * A * phi**2, (x, 0, L))

# ------------------------------------------------------------------
# 4. Kinetic Energy (Motor – lumped mass)
# ------------------------------------------------------------------
M11_m = m_m * x_m**2
M12_m = m_m * x_m * phi.subs(x, x_m)
M22_m = m_m * phi.subs(x, x_m)**2

# Total mass matrix
M = sp.Matrix([
    [M11_b + M11_m, M12_b + M12_m],
    [M12_b + M12_m, M22_b + M22_m]
])

# ------------------------------------------------------------------
# 5. Potential Energy
# ------------------------------------------------------------------

# Spring potential energy (at x = L)
K_s = k * sp.Matrix([
    [L**2, L * phi.subs(x, L)],
    [L * phi.subs(x, L), phi.subs(x, L)**2]
])

# Beam bending stiffness (only affects bending DOF)
K_b = sp.Matrix([
    [0, 0],
    [0, EI * sp.integrate(sp.diff(phi, x, 2)**2, (x, 0, L))]
])

# Total stiffness matrix
K = K_s + K_b

# ------------------------------------------------------------------
# 6. Equations of motion (matrix form)
# M q_dd + K q = 0
# ------------------------------------------------------------------
q_dd = sp.Matrix([theta_dd, eta_dd])
q = sp.Matrix([theta, eta])

EOM = M * q_dd + K * q

# ------------------------------------------------------------------
# 7. Eigenvalue problem for natural frequencies
# det(K - ω² M) = 0
# ------------------------------------------------------------------
omega2 = sp.symbols('omega2')

char_eq = sp.factor((K - omega2 * M).det())

# ------------------------------------------------------------------
# 8. Display results
# ------------------------------------------------------------------
print("Mass Matrix M:")
sp.pprint(M)

print("\nStiffness Matrix K:")
sp.pprint(K)

print("\nCharacteristic Equation det(K - ω²M) = 0:")
sp.pprint(char_eq)


Mass Matrix M:
⎡          3                          5                       ⎤
⎢       A⋅L ⋅ρ        2         11⋅A⋅L ⋅ρ        3            ⎥
⎢       ────── + mₘ⋅xₘ          ───────── + mₘ⋅xₘ ⋅(3⋅L - xₘ) ⎥
⎢         3                        20                         ⎥
⎢                                                             ⎥
⎢      5                              7                       ⎥
⎢11⋅A⋅L ⋅ρ        3             33⋅A⋅L ⋅ρ        4           2⎥
⎢───────── + mₘ⋅xₘ ⋅(3⋅L - xₘ)  ───────── + mₘ⋅xₘ ⋅(3⋅L - xₘ) ⎥
⎣   20                             35                         ⎦

Stiffness Matrix K:
⎡  2             4        ⎤
⎢ L ⋅k        2⋅L ⋅k      ⎥
⎢                         ⎥
⎢   4           3      6  ⎥
⎣2⋅L ⋅k  12⋅EI⋅L  + 4⋅L ⋅k⎦

Characteristic Equation det(K - ω²M) = 0:
  2 ⎛      2  8   2  2               4               7                  5      ↪
-L ⋅⎝- 99⋅A ⋅L ⋅ω₂ ⋅ρ  + 33600⋅A⋅EI⋅L ⋅ω₂⋅ρ + 640⋅A⋅L ⋅k⋅ω₂⋅ρ - 7920⋅A⋅L ⋅mₘ⋅ω ↪
───────────────────────────────────────────

## **Activity 1**

Modelling Assumptions:

For small-amplitude vibration,

1.Beam undergoes:


*   Small rigid rotation about the pinned end
*   Small elastic bending deformation


2.Linear spring behaves elastically (Hooke\textquotesingle s law).

3.Motor unbalance provides harmonic excitation (may be neglected for free vibration).

4.Damping is neglected for modal analysis.

5.Euler- Bernoulli beam theory applies.


Define two generalized coordinates:

$q_1(t) = \theta(t)$
→ Rigid-body rotation of the beam about the pinned end

$q_2(t) = \eta(t) $
→ Modal coordinate of the first bending mode

The transverse displacement of the beam at position \( x \) is approximated using assumed modes:

$$w(x,t)=x\theta(t)+\phi_1(x)\eta(t)$$

Where:
$xθ(t)$: rigid-body rotation displacement

$𝜙1(𝑥)$: first bending mode shape (satisfying pinned–spring boundary conditions)


The total kinetic energy consists of:

(a) Beam kinetic energy

$T_b = \frac{1}{2} \int_0^L \rho A (\dot{w}(x,t))^2 \, dx$

Substitute
$\dot{w}(x,t)$:
$w˙=xθ˙+ϕ_1(x)η˙$

After integration:

$$T_b = \frac{1}{2} \begin{bmatrix} \dot{\theta} \\ \dot{\eta} \end{bmatrix}^T \begin{bmatrix} I_b & M_{12} \\ M_{12} & M_{22} \end{bmatrix} \begin{bmatrix} \dot{\theta} \\ \dot{\eta} \end{bmatrix}$$

Where:
$$I_b = \int_0^L \rho A x^2 dx$$→ beam mass moment of inertia

$M_{12}$ → inertial coupling term

$M_{22}$ → modal mass of first bending mode

(b) Motor kinetic energy

Motor mass $m_{m}$ at position $x_{m}$:
$$T_m = \frac{1}{2} m_m (x_m \dot{\theta} + \phi_1(x_m) \dot{\eta})^2$$

This contributes additional terms to the mass matrix.

Potential Energy
𝑉

(a) Spring potential energy

Spring extension at free end:

$$\delta s = L\theta + \phi \frac{1}{L}\eta$$


$V_s = \frac{1}{2} k (L\theta + \phi_1(L)\eta)^2$

(b) Beam bending strain energy




